In [251]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt

In [252]:
def get_ohlcv(ticker, start=None, end=None, interval="1d"):
    data = yf.download(
        ticker,
        start=start,
        end=end,
        interval=interval,
        auto_adjust=False,
        progress=False
    )
    return data[["Open", "High", "Low", "Close", "Volume"]].dropna()

data = get_ohlcv("TCS.NS", "2023-01-01", "2024-01-01")
data = data.reset_index(drop=True)
data.columns = data.columns.get_level_values(0)

In [253]:
def backtest_engine(df):
    INITIAL_CAPITAL = 100000

    STOP_LOSS = 0.02      # 2% stop loss
    TAKE_PROFIT = 0.05   # 5% take profit

    capital = INITIAL_CAPITAL
    in_trade = False
    entry_price = 0.0

    portfolio_values = [capital]
    trade_returns = []

    total_trades = 0
    winning_trades = 0
    losing_trades = 0

    for i in range(1, len(df)):
        price = df['Close'].iloc[i]

        macd_now = df['MACD'].iloc[i]
        signal_now = df['signal_line'].iloc[i]
        hist_now = df['Histogram'].iloc[i]
        k_now = df['%K'].iloc[i]

        # enter long
        if not in_trade:
            if (
                macd_now > signal_now and
                hist_now > 0 and
                k_now < 61
            ):
                entry_price = price
                in_trade = True

        # exit long
        elif in_trade:
            ret = (price - entry_price) / entry_price

            if (
                ret <= -STOP_LOSS or
                ret >= TAKE_PROFIT or
                macd_now < signal_now
            ):
                capital *= (1 + ret)
                portfolio_values.append(capital)
                trade_returns.append(ret)

                total_trades += 1
                if ret > 0:
                    winning_trades += 1
                else:
                    losing_trades += 1

                in_trade = False

    # closing open trades
    if in_trade:
        final_price = df['Close'].iloc[-1]
        ret = (final_price - entry_price) / entry_price

        capital *= (1 + ret)
        portfolio_values.append(capital)
        trade_returns.append(ret)

        total_trades += 1
        if ret > 0:
            winning_trades += 1
        else:
            losing_trades += 1

    # metrics
    portfolio_values = np.array(portfolio_values)
    net_profit = capital - INITIAL_CAPITAL

    returns = np.diff(portfolio_values) / portfolio_values[:-1]
    if len(returns) > 1 and np.std(returns) != 0:
        sharpe_ratio = np.mean(returns) / np.std(returns) * np.sqrt(252)
    else:
        sharpe_ratio = 0.0

    cumulative_max = np.maximum.accumulate(portfolio_values)
    drawdown = (portfolio_values - cumulative_max) / cumulative_max
    max_drawdown = drawdown.min()

    print("Initial Capital:", INITIAL_CAPITAL)
    print("Final Capital:", round(capital, 2))
    print("Net Profit:", round(net_profit, 2))
    print("Sharpe Ratio:", round(sharpe_ratio, 2))
    print("Maximum Drawdown:", round(max_drawdown * 100, 2), "%")
    print("Total Trades:", total_trades)
    print("Winning Trades:", winning_trades)
    print("Losing Trades:", losing_trades)


In [254]:
def STOCHO(df):
  low_14 = df['Low'].rolling(window = 14).min()
  high_14 = df['High'].rolling(window = 14).max()

  df['%K'] = 100 * ((df['Close'] - low_14) / (high_14 - low_14))
  df['%D'] = df['%K'].rolling(3).mean()

  return df

def macd(df,short_period,long_period,p):
  df['long_ema'] = df['Close'].ewm(span=long_period,adjust=False).mean()
  df['short_ema'] = df['Close'].ewm(span=short_period,adjust=False).mean()
  df['MACD'] = df['short_ema'] - df['long_ema']
  df['signal_line'] = df['MACD'].ewm(span=p,adjust=False).mean()
  df['Histogram'] = df['MACD'] - df['signal_line']

  return df

data = macd(data,12,26,9)
data = STOCHO(data)
data.dropna(inplace = True)


In [255]:
backtest_engine(data)

Initial Capital: 100000
Final Capital: 123984.15
Net Profit: 23984.15
Sharpe Ratio: 10.62
Maximum Drawdown: -3.87 %
Total Trades: 9
Winning Trades: 6
Losing Trades: 3
